#Excel ETL Project

##Extract Part

In [ ]:
import sqlite3
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


##Create Data Base
db_path = "/content/drive/MyDrive/ETL-Excel-Loan-Project/loans.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
csv_path = "/content/drive/MyDrive/ETL-Excel-Loan-Project/data/loan_final313_.csv"
df = pd.read_csv(csv_path)

# Load into SQL table
df.to_sql('loans', conn, if_exists='replace', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


70000

## Data Verification


In [ ]:
pd.read_sql_query("SELECT COUNT(*) FROM loans", conn)

,COUNT(*)
0,70000


In [ ]:
pd.set_option('display.max_columns',None)
pd.read_sql_query("SELECT * FROM loans LIMIT 10",conn)


,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,loan_amount,term,term_cat,application_type,application_type_cat,purpose,purpose_cat,interest_payments,interest_payment_cat,loan_condition,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,6297794,2013,01-07-2013,1122015,10.0,MORTGAGE,3,Low,55000,1,10000,36 months,1,INDIVIDUAL,1,debt_consolidation,6,Low,1,Good Loan,0,10.64,B,2,20.57,11630.35000,10000.00,0.0,325.69,cannught
1,4526163,2013,01-05-2013,1102014,9.0,RENT,1,Low,60000,1,10575,36 months,1,INDIVIDUAL,1,home_improvement,7,High,2,Good Loan,0,21.00,E,5,8.82,13165.83707,10575.00,0.0,398.42,cannught
2,9001808,2013,01-11-2013,1012016,10.0,MORTGAGE,3,Low,63000,1,8875,36 months,1,INDIVIDUAL,1,debt_consolidation,6,High,2,Good Loan,0,13.67,B,2,8.72,7849.66000,6033.91,0.0,301.91,munster
3,7306367,2013,01-09-2013,1122015,10.0,MORTGAGE,3,Low,100000,1,10500,60 months,2,INDIVIDUAL,1,major_purchase,8,High,2,Good Loan,0,16.20,C,3,19.52,6924.42000,3707.15,0.0,256.46,ulster
4,6532028,2013,01-08-2013,1082014,10.0,MORTGAGE,3,Low,85000,1,24000,36 months,1,INDIVIDUAL,1,debt_consolidation,6,High,2,Good Loan,0,16.78,C,3,15.49,27523.60099,24000.00,0.0,853.04,ulster
5,6316917,2013,01-07-2013,1102014,1.0,RENT,1,Low,82500,1,9000,36 months,1,INDIVIDUAL,1,debt_consolidation,6,Low,1,Good Loan,0,9.71,B,2,7.96,9901.40164,9000.00,0.0,289.19,munster
6,1570001,2012,01-10-2012,1102015,0.5,RENT,1,Low,55000,1,12000,36 months,1,INDIVIDUAL,1,debt_consolidation,6,High,2,Good Loan,0,15.31,C,3,24.34,15040.97044,12000.00,0.0,417.81,Northern-Irl
7,26199528,2014,01-10-2014,1012016,10.0,RENT,1,Low,67980,1,10000,36 months,1,INDIVIDUAL,1,debt_consolidation,6,Low,1,Good Loan,0,6.03,A,1,6.54,4761.17000,4158.69,0.0,304.36,Northern-Irl
8,35074071,2014,01-11-2014,1122015,10.0,RENT,1,Low,51000,1,7000,36 months,1,INDIVIDUAL,1,credit_card,1,Low,1,Good Loan,0,8.67,B,2,12.40,2876.52000,2321.36,0.0,221.53,Northern-Irl
9,3725985,2013,01-03-2013,1012016,1.0,MORTGAGE,3,Low,40000,1,17875,60 months,2,INDIVIDUAL,1,credit_card,1,High,2,Good Loan,0,16.29,C,3,20.73,14435.85000,8041.99,0.0,437.45,ulster


#Checking Data Inconsistencies

###Checking For NULL values

In [ ]:
df.isnull().sum()

,0
id,0
year,0
issue_d,0
final_d,0
emp_length_int,0
home_ownership,0
home_ownership_cat,0
income_category,0
annual_inc,0
income_cat,0


In [ ]:
#Checking Null Values in Important columns
query=""" SELECT
sum(case when loan_amount is NULL or loan_amount="" then 1 else 0 END) as loan_amount_NULLs,
sum(case when id is NULL or id="" then 1 else 0 END)as id_NULLs ,
sum(case when interest_rate is NULL or interest_rate="" then 1 else 0 END) as interest_rate_NULLs,
sum(case when annual_inc is NULL or annual_inc="" then 1 else 0 END)as annual_inc_NULLs,
sum(case when loan_condition is NULL or loan_condition ="" then 1 else 0 END)as loan_condition_NULLs FROM loans;"""

pd.read_sql_query(query,conn)

,loan_amount_NULLs,id_NULLs,interest_rate_NULLs,annual_inc_NULLs,loan_condition_NULLs
0,0,0,0,0,0


From Above results we can clearly see that there no NULL Values present in any of our critical columns so do not need to impute null values.

### Checking for Duplicates Values

In [ ]:
## For Checking Duplicated entries we just need to check 'id' columns
query=""" Select id,count(*) from loans
group by id
having count(*)>1"""

pd.read_sql_query(query,conn)

,id,count(*)


Above result shows that out dataset also don't have any duplicate values.

##Check all unique values in all Categorical Columns

In [ ]:
queries=["""SELECT distinct year from loans""",""" SELECT DISTINCT home_ownership from loans""","""SELECT DISTINCT income_category from loans""","""SELECT DISTINCT term from loans""","""SELECT DISTINCT application_type from loans""",
         """SELECT DISTINCT purpose from loans""","""SELECT DISTINCT interest_payments from loans""","""SELECT DISTINCT loan_condition from loans""","""SELECT DISTINCT grade from loans""","""SELECT DISTINCT region from loans"""]


for query in queries:
  print(pd.read_sql_query(query,conn))
  print("")

   year
0  2013
1  2012
2  2014
3  2010
4  2011
5  2009
6  2008
7  2007

  home_ownership
0       MORTGAGE
1           RENT
2            OWN
3          OTHER
4           NONE

  income_category
0             Low
1          Medium
2            High

         term
0   36 months
1   60 months

  application_type
0       INDIVIDUAL

               purpose
0   debt_consolidation
1     home_improvement
2       major_purchase
3          credit_card
4              medical
5                  car
6                other
7              wedding
8               moving
9       small_business
10               house
11            vacation
12    renewable_energy
13         educational

  interest_payments
0               Low
1              High

  loan_condition
0      Good Loan
1       Bad Loan

  grade
0     B
1     E
2     C
3     A
4     G
5     D
6     F

         region
0      cannught
1       munster
2        ulster
3  Northern-Irl
4      leinster



##Tranformation Phase

###Standardize Categories

In [ ]:
#Standardize text / categorical columns (UPPER + TRIM) and spelling checks

sql=""" UPDATE loans
SET income_category=UPPER(TRIM(income_category)),
purpose=UPPER(TRIM(purpose)),
interest_payments=UPPER(TRIM(interest_payments)),
loan_condition=UPPER(TRIM(loan_condition)),
region=UPPER(TRIM(region));
UPDATE loans SET region = 'CONNAUGHT' WHERE region = 'CANNUUGHT' OR region = 'CANNUAGHT';
UPDATE loans SET region = 'NORTHERN IRL' WHERE region IN ('NORTHERN-IRL','NORTHERN_IRL');"""

"""Update home_ownership NONE Category to OTHER Category Beacause
“In the raw dataset, HOME_OWNERSHIP had a rare category ‘NONE’.
This category has no analytical relevance and is very small,
which can lead to cluttered pivot charts. To simplify visualizations and maintain consistent housing categories,
I merged ‘NONE’ into ‘OTHER’. This is a common data-cleaning practice in real-world analytics to improve interpretability.”
"""


cursor.execute("""
UPDATE loans
SET home_ownership = 'OTHER'
WHERE home_ownership = 'NONE';
""")
conn.commit()

conn.executescript(sql)
conn.commit()

queries=["""SELECT distinct year from loans""",""" SELECT DISTINCT home_ownership from loans""","""SELECT DISTINCT income_category from loans""","""SELECT DISTINCT term from loans""","""SELECT DISTINCT application_type from loans""",
         """SELECT DISTINCT purpose from loans""","""SELECT DISTINCT interest_payments from loans""","""SELECT DISTINCT loan_condition from loans""","""SELECT DISTINCT grade from loans""","""SELECT DISTINCT region from loans"""]


for query in queries:
  print(pd.read_sql_query(query,conn))
  print("")


   year
0  2013
1  2012
2  2014
3  2010
4  2011
5  2009
6  2008
7  2007

  home_ownership
0       MORTGAGE
1           RENT
2            OWN
3          OTHER

  income_category
0             LOW
1          MEDIUM
2            HIGH

         term
0   36 months
1   60 months

  application_type
0       INDIVIDUAL

               purpose
0   DEBT_CONSOLIDATION
1     HOME_IMPROVEMENT
2       MAJOR_PURCHASE
3          CREDIT_CARD
4              MEDICAL
5                  CAR
6                OTHER
7              WEDDING
8               MOVING
9       SMALL_BUSINESS
10               HOUSE
11            VACATION
12    RENEWABLE_ENERGY
13         EDUCATIONAL

  interest_payments
0               LOW
1              HIGH

  loan_condition
0      GOOD LOAN
1       BAD LOAN

  grade
0     B
1     E
2     C
3     A
4     G
5     D
6     F

         region
0      CANNUGHT
1       MUNSTER
2        ULSTER
3  NORTHERN IRL
4      LEINSTER



In [ ]:
sql = """

-- Convert issue_d: DD-MM-YYYY -> YYYY-MM-DD

UPDATE loans
SET issue_d = printf('%04d-%02d-%02d',
                     CAST(substr(issue_d, 7, 4) AS INTEGER),
                     CAST(substr(issue_d, 4, 2) AS INTEGER),
                     CAST(substr(issue_d, 1, 2) AS INTEGER))
WHERE issue_d GLOB '[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]';


-- Convert final_d: always DMMYYYY -> YYYY-MM-01

UPDATE loans
SET final_d = printf('%04d-%02d-%02d',
                     CAST(substr(final_d, 4, 4) AS INTEGER),   -- year
                     CAST(substr(final_d, 2, 2) AS INTEGER),   -- month
                     1)                                        -- day = always 1
WHERE LENGTH(final_d) = 7
  AND final_d GLOB '[0-9]*';


-- Canonicalize using SQLite date()

UPDATE loans SET issue_d = date(issue_d);
UPDATE loans
SET final_d = TRIM(final_d);
UPDATE loans set final_d=date(final_d);
"""

cursor.executescript(sql)
conn.commit()

print("Date standardization complete!")


Date standardization complete!


In [ ]:
display(pd.read_sql_query("""
SELECT id, issue_d, final_d
FROM loans
LIMIT 20;
""", conn))

,id,issue_d,final_d
0,6297794,2013-07-01,2015-12-01
1,4526163,2013-05-01,2014-10-01
2,9001808,2013-11-01,2016-01-01
3,7306367,2013-09-01,2015-12-01
4,6532028,2013-08-01,2014-08-01
5,6316917,2013-07-01,2014-10-01
6,1570001,2012-10-01,2015-10-01
7,26199528,2014-10-01,2016-01-01
8,35074071,2014-11-01,2015-12-01
9,3725985,2013-03-01,2016-01-01


###Feature Engineering

In [ ]:
## Create New columns profitability risk flag
sql="""
alter table loans add profitability real;
UPDATE loans set profitability=total_pymnt-loan_amount;

alter table loans add risk_flag INTEGER;
UPDATE loans set risk_flag= CASE when loan_condition="GOOD LOAN" then 0 else 1 END;

alter table loans add income_profit_ratio REAL;
UPDATE loans
set income_profit_ratio=annual_inc*1.0/loan_amount;

alter table loans add default_rate_indicator real;

with yrbad as (
  select year, sum(case when loan_condition='BAD LOAN' then 1 else 0 end) as bad,
  count(*) as total
  from loans
  group by year
)
UPDATE loans
set default_rate_indicator=(select (bad*1.0/total) from yrbad where loans.year=yrbad.year);
"""

cursor.executescript(sql)
conn.commit()

In [ ]:
# Extract loan term as numeric value
sql="""ALTER TABLE loans ADD COLUMN term_months INTEGER;

UPDATE loans
SET term_months = CAST(REPLACE(term, ' months', '') AS INTEGER);"""

cursor.executescript(sql)
conn.commit()






In [ ]:
query=""" select * from loans limit 20"""
pd.read_sql_query(query,conn)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,loan_amount,term,term_cat,application_type,application_type_cat,purpose,purpose_cat,interest_payments,interest_payment_cat,loan_condition,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region,profitability,risk_flag,income_profit_ratio,default_rate_indicator,term_months
0,6297794,2013,2013-07-01,2015-12-01,10.0,MORTGAGE,3,LOW,55000,1,10000,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,LOW,1,GOOD LOAN,0,10.64,B,2,20.57,11630.350000,10000.00,0.00,325.69,CANNUGHT,1630.350000,0,5.500000,0.138831,36
1,4526163,2013,2013-05-01,2014-10-01,9.0,RENT,1,LOW,60000,1,10575,36 months,1,INDIVIDUAL,1,HOME_IMPROVEMENT,7,HIGH,2,GOOD LOAN,0,21.00,E,5,8.82,13165.837070,10575.00,0.00,398.42,CANNUGHT,2590.837070,0,5.673759,0.138831,36
2,9001808,2013,2013-11-01,2016-01-01,10.0,MORTGAGE,3,LOW,63000,1,8875,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,HIGH,2,GOOD LOAN,0,13.67,B,2,8.72,7849.660000,6033.91,0.00,301.91,MUNSTER,-1025.340000,0,7.098592,0.138831,36
3,7306367,2013,2013-09-01,2015-12-01,10.0,MORTGAGE,3,LOW,100000,1,10500,60 months,2,INDIVIDUAL,1,MAJOR_PURCHASE,8,HIGH,2,GOOD LOAN,0,16.20,C,3,19.52,6924.420000,3707.15,0.00,256.46,ULSTER,-3575.580000,0,9.523810,0.138831,60
4,6532028,2013,2013-08-01,2014-08-01,10.0,MORTGAGE,3,LOW,85000,1,24000,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,HIGH,2,GOOD LOAN,0,16.78,C,3,15.49,27523.600990,24000.00,0.00,853.04,ULSTER,3523.600990,0,3.541667,0.138831,36
5,6316917,2013,2013-07-01,2014-10-01,1.0,RENT,1,LOW,82500,1,9000,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,LOW,1,GOOD LOAN,0,9.71,B,2,7.96,9901.401640,9000.00,0.00,289.19,MUNSTER,901.401640,0,9.166667,0.138831,36
6,1570001,2012,2012-10-01,2015-10-01,0.5,RENT,1,LOW,55000,1,12000,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,HIGH,2,GOOD LOAN,0,15.31,C,3,24.34,15040.970440,12000.00,0.00,417.81,NORTHERN IRL,3040.970440,0,4.583333,0.159463,36
7,26199528,2014,2014-10-01,2016-01-01,10.0,RENT,1,LOW,67980,1,10000,36 months,1,INDIVIDUAL,1,DEBT_CONSOLIDATION,6,LOW,1,GOOD LOAN,0,6.03,A,1,6.54,4761.170000,4158.69,0.00,304.36,NORTHERN IRL,-5238.830000,0,6.798000,0.079046,36
8,35074071,2014,2014-11-01,2015-12-01,10.0,RENT,1,LOW,51000,1,7000,36 months,1,INDIVIDUAL,1,CREDIT_CARD,1,LOW,1,GOOD LOAN,0,8.67,B,2,12.40,2876.520000,2321.36,0.00,221.53,NORTHERN IRL,-4123.480000,0,7.285714,0.079046,36
9,3725985,2013,2013-03-01,2016-01-01,1.0,MORTGAGE,3,LOW,40000,1,17875,60 months,2,INDIVIDUAL,1,CREDIT_CARD,1,HIGH,2,GOOD LOAN,0,16.29,C,3,20.73,14435.850000,8041.99,0.00,437.45,ULSTER,-3439.150000,0,2.237762,0.138831,60


##Loading Phase

In [ ]:
# Read the cleaned table from SQLite
df_clean = pd.read_sql_query("SELECT * FROM loans", conn)

# Define output path
output_path = "/content/drive/MyDrive/ETL-Excel-Loan-Project/data/loans_cleaned.csv"

# Save as CSV
df_clean.to_csv(output_path, index=False)

In [ ]:
query=""" select year,sum(risk_flag) from loans group by year"""
pd.read_sql_query(query,conn)

,year,sum(risk_flag)
0,2007,44
1,2008,99
2,2009,170
3,2010,422
4,2011,716
5,2012,1878
6,2013,4123
7,2014,1512
